## 导入tensorflow模块并启用急切执行 
Eager Excution可以为TensorFlow提供更具交互性的前端

In [1]:
import tensorflow as tf
tf.enable_eager_execution()
tf.__version__

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.10.0'

## Tensor
张量是一个多维数组。 与NumPy ndarray对象类似，Tensor对象具有数据类型和形状。 此外，Tensors可以驻留在加速器（如GPU）内存中。 TensorFlow提供了丰富的操作库（tf.add，tf.matmul，tf.linalg.inv等），它们使用和生成Tensors。 这些操作自动转换本机Python类型。 例如：

In [2]:
print(tf.add(1,2))
print(tf.add([1,2],[3,4]))
print(tf.square(5))
print(tf.reduce_sum([1,2,3]))
print(tf.encode_base64("hello world"))

print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG8gd29ybGQ', shape=(), dtype=string)
tf.Tensor(13, shape=(), dtype=int32)


In [3]:
x = tf.matmul([[1]],[[2,3]])
print(x.shape)
print(x.dtype)

(1, 2)
<dtype: 'int32'>


NumPy兼容性

TensorFlow张量和NumPy nararrays之间的转换非常简单，如：

- TensorFlow操作自动将NumPy ndarrays转换为Tensors。
- NumPy操作自动将Tensors转换为NumPy ndarrays。

通过在它们上调用.numpy（）方法，可以将张量显式转换为NumPy ndarrays。 这些转换通常很便宜，因为如果可能，数组和Tensor共享底层内存表示。 但是，共享底层表示并不总是可行的，因为Tensor可能托管在GPU内存中，而NumPy阵列总是由主机内存支持，因此转换将涉及从GPU到主机内存的复制。

In [4]:
import numpy as np
ndarray = np.ones([3,3])

print("Tensorflow operation convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray,42)
print(tensor)

print("And Numpy operations convert Tensors to numpy array automatically")
print(np.add(tensor,1))

print("The.numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

Tensorflow operation convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And Numpy operations convert Tensors to numpy array automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The.numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


## GPU 加速

通过使用GPU进行计算，可以加速许多TensorFlow操作。 在没有任何注释的情况下，TensorFlow会自动决定是使用GPU还是CPU进行操作（如有必要，还可以复制CPU和GPU内存之间的张量）。 由操作产生的张量通常由执行操作的设备的存储器支持。 例如：

In [5]:
x = tf.random_uniform([3,3])

print("Is There a GPU avaliable:")
print(tf.test.is_gpu_available())

print("Is the Tensor on GPU #0: ")
print(x.device.endswith('GPU:0'))

Is There a GPU avaliable:
False
Is the Tensor on GPU #0: 
False


## 设备名称
Tensor.device属性提供托管Tensor内容的设备的完全限定字符串名称。 此名称对一组详细信息进行编码，例如，正在执行此程序的主机的网络地址的标识符以及该主机中的设备。 这是分布式执行TensorFlow程序所必需的，但我们暂时不会这样做。 如果张量位于主机上的第N个张量上，则字符串将以GPU结尾：<N>。

## 显式设备放置

TensorFlow中的术语“放置”指的是如何为执行设备分配（放置）各个操作。 如上所述，当没有提供明确的指导时，TensorFlow会自动决定执行操作的设备，并在需要时将Tensors复制到该设备。 但是，可以使用tf.device上下文管理器将TensorFlow操作显式放置在特定设备上。 例如：

In [ ]:
def time_matmul(x):
    %timeit tf.matmul(x,x)

# Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
    x = tf.random_uniform([1000,1000])
    assert x.device.endswith("CPU:0")
    time_matmul(x)

# Force execution on GPU #0 if available
if tf.test.is_gpu_available():
    with tf.device("GPU:0"):
        x = tf.random_uniform([1000,1000])
        assert x.device.endswith("GPU:0")
        time_matmul(x)

On CPU:


## Datasets

本节演示如何使用tf.data.Dataset API构建管道以将数据提供给模型。 它涵盖：

- 创建数据集。
- 在启用了急切执行的情况下对数据集进行迭代。

我们建议使用数据集API从简单，可重复使用的部分构建高性能，复杂的输入管道，这些部分将为模型的培训或评估循环提供支持。

如果您熟悉TensorFlow图，则在启用eager执行时，构建数据集对象的API保持完全相同，但迭代数据集元素的过程稍微简单一些。 您可以对tf.data.Dataset对象使用Python迭代，而不需要显式创建tf.data.Iterator对象。 因此，在启用eager执行时，TensorFlow指南中对迭代器的讨论无关紧要。

## Create a source Dataset

使用其中一个工厂函数（如Dataset.from_tensors，Dataset.from_tensor_slices）或使用从TextLineDataset或TFRecordDataset等文件读取的对象创建源数据集。 有关更多信息，请参阅TensorFlow指南。

In [ ]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,6])

#Create a CSV file
import tempfile
_,filename = tempfile.mkstemp()

with open(filename,'w') as f:
    f.write("""Line 1
    Line 2
    Line 3
    """)
ds_file = tf.data.TextLineDataset(filename)

## 应用转换
使用map，batch，shuffle等转换函数将转换应用于数据集的记录。 有关详细信息，请参阅tf.data.Dataset的API文档。

In [ ]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

## 迭代
启用eager执行时，Dataset对象支持迭代。 如果您熟悉TensorFlow图中数据集的使用，请注意不需要调用Dataset.make_one_shot_iterator（）或get_next（）调用。

In [ ]:
print('Elements of ds_tensors:')
for x in ds_tensors:
    print(x.numpy())

print("\nElements in ds_file:")
for x in ds_file:
    print(x.numpy())